In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
class Arg:
    arch = 'resnet18'
    #data = '/data/imagenet/ILSVRC/Data/CLS-LOC/'
    data = '/home/jtang/Desktop'
    batch_size = 128
    workers = 4
    start_epoch = 0
    epochs = 90
    lr = 0.1
    momentum = 0.9
    weight_decay = 1e-4
    print_freq = 10
    resume = ''
    world_size = -1
    rank = -1
    dist_url = 'tcp://224.66.41.62:23456'
    seed = None
    dist_backend = 'nccl'
    gpu = None
    distributed = False
    pretrained = False
    evaluate = False
    multiprocessing_distributed = False
args = Arg()

In [3]:
best_acc1 = 0

In [4]:
ngpus_per_node = torch.cuda.device_count()

In [5]:
def train(train_loader, model, criterion, optimizer, epoch, args):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if args.gpu is not None:
            images = images.cuda(args.gpu, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(args.gpu, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            progress.display(i)


def validate(val_loader, model, criterion, args):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if args.gpu is not None:
                images = images.cuda(args.gpu, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(args.gpu, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [6]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [7]:
def adjust_learning_rate(optimizer, epoch, args):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [8]:
def main_worker(gpu, ngpus_per_node, args):
    global best_acc1
    args.gpu = gpu

    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))

    if args.distributed:
        if args.dist_url == "env://" and args.rank == -1:
            args.rank = int(os.environ["RANK"])
        if args.multiprocessing_distributed:
            # For multiprocessing distributed training, rank needs to be the
            # global rank among all the processes
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                world_size=args.world_size, rank=args.rank)
    # create model
    if args.pretrained:
        print("=> using pre-trained model '{}'".format(args.arch))
        model = models.__dict__[args.arch](pretrained=True)
    else:
        print("=> creating model '{}'".format(args.arch))
        model = models.__dict__[args.arch]()

    if not torch.cuda.is_available():
        print('using CPU, this will be slow')
    elif args.distributed:
        # For multiprocessing distributed, DistributedDataParallel constructor
        # should always set the single device scope, otherwise,
        # DistributedDataParallel will use all available devices.
        if args.gpu is not None:
            torch.cuda.set_device(args.gpu)
            model.cuda(args.gpu)
            # When using a single GPU per process and per
            # DistributedDataParallel, we need to divide the batch size
            # ourselves based on the total number of GPUs we have
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int((args.workers + ngpus_per_node - 1) / ngpus_per_node)
            model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        else:
            model.cuda()
            # DistributedDataParallel will divide and allocate batch_size to all
            # available GPUs if device_ids are not set
            model = torch.nn.parallel.DistributedDataParallel(model)
    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        model = model.cuda(args.gpu)
    else:
        # DataParallel will divide and allocate batch_size to all available GPUs
        if args.arch.startswith('alexnet') or args.arch.startswith('vgg'):
            model.features = torch.nn.DataParallel(model.features)
            model.cuda()
        else:
            model = torch.nn.DataParallel(model).cuda()

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda(args.gpu)

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            if args.gpu is None:
                checkpoint = torch.load(args.resume)
            else:
                # Map model to be loaded to specified single gpu.
                loc = 'cuda:{}'.format(args.gpu)
                checkpoint = torch.load(args.resume, map_location=loc)
            args.start_epoch = checkpoint['epoch']
            best_acc1 = checkpoint['best_acc1']
            if args.gpu is not None:
                # best_acc1 may be from a checkpoint from a different GPU
                best_acc1 = best_acc1.to(args.gpu)
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True

    # Data loading code
    # /data/imagenet_augment_contrast/contrast
    traindir = '/data/imagenet_augment_contrast/contrast/2'
    valdir = os.path.join(args.data, 'val')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_dataset = datasets.ImageFolder(
        traindir,
        transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    else:
        train_sampler = None

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=(train_sampler is None),
        num_workers=args.workers, pin_memory=True, sampler=train_sampler)

    val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=True)

    if args.evaluate:
        validate(val_loader, model, criterion, args)
        return

    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            train_sampler.set_epoch(epoch)
        adjust_learning_rate(optimizer, epoch, args)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch, args)

        # evaluate on validation set
        acc1 = validate(val_loader, model, criterion, args)

        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        if not args.multiprocessing_distributed or (args.multiprocessing_distributed
                and args.rank % ngpus_per_node == 0):
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': args.arch,
                'state_dict': model.state_dict(),
                'best_acc1': best_acc1,
                'optimizer' : optimizer.state_dict(),
            }, is_best)

In [ ]:
main_worker(args.gpu, ngpus_per_node, args)

=> creating model 'resnet18'
Epoch: [0][   0/1002]	Time  3.555 ( 3.555)	Data  0.877 ( 0.877)	Loss 7.0590e+00 (7.0590e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.78 (  0.78)
Epoch: [0][  10/1002]	Time  0.318 ( 0.602)	Data  0.000 ( 0.081)	Loss 7.5019e+00 (7.1561e+00)	Acc@1   0.78 (  0.28)	Acc@5   0.78 (  0.71)
Epoch: [0][  20/1002]	Time  0.320 ( 0.474)	Data  0.000 ( 0.053)	Loss 7.3389e+00 (7.2270e+00)	Acc@1   0.00 (  0.41)	Acc@5   0.00 (  0.86)
Epoch: [0][  30/1002]	Time  0.325 ( 0.427)	Data  0.000 ( 0.043)	Loss 7.0926e+00 (7.2063e+00)	Acc@1   0.00 (  0.33)	Acc@5   0.78 (  0.86)
Epoch: [0][  40/1002]	Time  0.335 ( 0.405)	Data  0.000 ( 0.038)	Loss 6.9626e+00 (7.1639e+00)	Acc@1   0.00 (  0.29)	Acc@5   0.78 (  0.76)
Epoch: [0][  50/1002]	Time  0.361 ( 0.392)	Data  0.000 ( 0.035)	Loss 7.0138e+00 (7.1493e+00)	Acc@1   0.00 (  0.25)	Acc@5   0.00 (  0.72)
Epoch: [0][  60/1002]	Time  0.353 ( 0.382)	Data  0.000 ( 0.033)	Loss 6.9151e+00 (7.1190e+00)	Acc@1   0.00 (  0.23)	Acc@5   0.00 (  0.70)
Epoch: [0][ 

Epoch: [0][ 600/1002]	Time  0.350 ( 0.350)	Data  0.000 ( 0.025)	Loss 6.3893e+00 (6.7185e+00)	Acc@1   2.34 (  0.57)	Acc@5   2.34 (  2.24)
Epoch: [0][ 610/1002]	Time  0.346 ( 0.351)	Data  0.000 ( 0.025)	Loss 6.5513e+00 (6.7144e+00)	Acc@1   0.00 (  0.57)	Acc@5   0.78 (  2.27)
Epoch: [0][ 620/1002]	Time  0.348 ( 0.351)	Data  0.000 ( 0.025)	Loss 6.3963e+00 (6.7098e+00)	Acc@1   1.56 (  0.58)	Acc@5   5.47 (  2.29)
Epoch: [0][ 630/1002]	Time  0.359 ( 0.351)	Data  0.000 ( 0.025)	Loss 6.2997e+00 (6.7052e+00)	Acc@1   0.78 (  0.58)	Acc@5   5.47 (  2.32)
Epoch: [0][ 640/1002]	Time  0.359 ( 0.351)	Data  0.000 ( 0.025)	Loss 6.3914e+00 (6.7017e+00)	Acc@1   0.78 (  0.58)	Acc@5   5.47 (  2.34)
Epoch: [0][ 650/1002]	Time  0.358 ( 0.351)	Data  0.000 ( 0.025)	Loss 6.5995e+00 (6.6979e+00)	Acc@1   1.56 (  0.59)	Acc@5   1.56 (  2.37)
Epoch: [0][ 660/1002]	Time  0.351 ( 0.351)	Data  0.000 ( 0.025)	Loss 6.4535e+00 (6.6950e+00)	Acc@1   1.56 (  0.60)	Acc@5   3.12 (  2.39)
Epoch: [0][ 670/1002]	Time  0.350 ( 0.351

Epoch: [1][ 180/1002]	Time  0.361 ( 0.371)	Data  0.000 ( 0.028)	Loss 6.0668e+00 (6.1714e+00)	Acc@1   1.56 (  1.75)	Acc@5   8.59 (  6.55)
Epoch: [1][ 190/1002]	Time  0.366 ( 0.370)	Data  0.000 ( 0.028)	Loss 6.1742e+00 (6.1693e+00)	Acc@1   0.78 (  1.76)	Acc@5   5.47 (  6.55)
Epoch: [1][ 200/1002]	Time  0.367 ( 0.370)	Data  0.000 ( 0.028)	Loss 6.2399e+00 (6.1633e+00)	Acc@1   3.12 (  1.78)	Acc@5   5.47 (  6.59)
Epoch: [1][ 210/1002]	Time  0.366 ( 0.370)	Data  0.000 ( 0.028)	Loss 6.1909e+00 (6.1600e+00)	Acc@1   3.12 (  1.79)	Acc@5   8.59 (  6.62)
Epoch: [1][ 220/1002]	Time  0.366 ( 0.370)	Data  0.000 ( 0.027)	Loss 6.1816e+00 (6.1588e+00)	Acc@1   3.91 (  1.82)	Acc@5   6.25 (  6.64)
Epoch: [1][ 230/1002]	Time  0.363 ( 0.370)	Data  0.000 ( 0.027)	Loss 6.1860e+00 (6.1570e+00)	Acc@1   1.56 (  1.84)	Acc@5   8.59 (  6.68)
Epoch: [1][ 240/1002]	Time  0.368 ( 0.370)	Data  0.000 ( 0.027)	Loss 6.0990e+00 (6.1532e+00)	Acc@1   2.34 (  1.85)	Acc@5   8.59 (  6.70)
Epoch: [1][ 250/1002]	Time  0.362 ( 0.370

Epoch: [1][ 780/1002]	Time  0.368 ( 0.369)	Data  0.000 ( 0.026)	Loss 5.9804e+00 (6.0174e+00)	Acc@1   2.34 (  2.47)	Acc@5   6.25 (  8.36)
Epoch: [1][ 790/1002]	Time  0.366 ( 0.370)	Data  0.000 ( 0.026)	Loss 5.8608e+00 (6.0143e+00)	Acc@1   3.12 (  2.48)	Acc@5   9.38 (  8.40)
Epoch: [1][ 800/1002]	Time  0.364 ( 0.370)	Data  0.000 ( 0.026)	Loss 5.7158e+00 (6.0116e+00)	Acc@1   3.91 (  2.49)	Acc@5  10.16 (  8.42)
Epoch: [1][ 810/1002]	Time  0.361 ( 0.370)	Data  0.000 ( 0.026)	Loss 5.5547e+00 (6.0083e+00)	Acc@1   3.91 (  2.49)	Acc@5  11.72 (  8.45)
Epoch: [1][ 820/1002]	Time  0.364 ( 0.370)	Data  0.000 ( 0.026)	Loss 6.0379e+00 (6.0063e+00)	Acc@1   1.56 (  2.50)	Acc@5   4.69 (  8.47)
Epoch: [1][ 830/1002]	Time  0.366 ( 0.370)	Data  0.000 ( 0.026)	Loss 5.8049e+00 (6.0037e+00)	Acc@1   2.34 (  2.51)	Acc@5  10.94 (  8.50)
Epoch: [1][ 840/1002]	Time  0.365 ( 0.370)	Data  0.000 ( 0.026)	Loss 5.8548e+00 (6.0013e+00)	Acc@1   1.56 (  2.52)	Acc@5  10.16 (  8.53)
Epoch: [1][ 850/1002]	Time  0.372 ( 0.370

Epoch: [2][ 360/1002]	Time  0.369 ( 0.371)	Data  0.000 ( 0.027)	Loss 5.4921e+00 (5.5799e+00)	Acc@1   4.69 (  4.72)	Acc@5  20.31 ( 14.08)
Epoch: [2][ 370/1002]	Time  0.367 ( 0.371)	Data  0.000 ( 0.027)	Loss 5.6000e+00 (5.5783e+00)	Acc@1   3.91 (  4.74)	Acc@5   8.59 ( 14.11)
Epoch: [2][ 380/1002]	Time  0.410 ( 0.371)	Data  0.000 ( 0.027)	Loss 5.4622e+00 (5.5764e+00)	Acc@1   3.91 (  4.77)	Acc@5  17.97 ( 14.15)
Epoch: [2][ 390/1002]	Time  0.367 ( 0.371)	Data  0.000 ( 0.027)	Loss 5.3445e+00 (5.5737e+00)	Acc@1   7.03 (  4.77)	Acc@5  21.09 ( 14.18)
Epoch: [2][ 400/1002]	Time  0.375 ( 0.371)	Data  0.000 ( 0.027)	Loss 5.7433e+00 (5.5751e+00)	Acc@1   4.69 (  4.76)	Acc@5  11.72 ( 14.13)
Epoch: [2][ 410/1002]	Time  0.365 ( 0.371)	Data  0.000 ( 0.027)	Loss 5.4535e+00 (5.5741e+00)	Acc@1   5.47 (  4.77)	Acc@5  15.62 ( 14.14)
Epoch: [2][ 420/1002]	Time  0.391 ( 0.371)	Data  0.000 ( 0.027)	Loss 5.4188e+00 (5.5731e+00)	Acc@1   5.47 (  4.77)	Acc@5  14.84 ( 14.15)
Epoch: [2][ 430/1002]	Time  0.369 ( 0.371

...

Epoch: [9][ 420/1002]	Time  0.364 ( 0.369)	Data  0.000 ( 0.027)	Loss 4.0723e+00 (3.8701e+00)	Acc@1  25.78 ( 22.59)	Acc@5  46.88 ( 45.03)
Epoch: [9][ 430/1002]	Time  0.368 ( 0.369)	Data  0.000 ( 0.026)	Loss 4.0994e+00 (3.8723e+00)	Acc@1  15.62 ( 22.54)	Acc@5  40.62 ( 45.01)
Epoch: [9][ 440/1002]	Time  0.385 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.9681e+00 (3.8760e+00)	Acc@1  23.44 ( 22.48)	Acc@5  40.62 ( 44.95)
Epoch: [9][ 450/1002]	Time  0.367 ( 0.369)	Data  0.000 ( 0.026)	Loss 4.0657e+00 (3.8771e+00)	Acc@1  15.62 ( 22.46)	Acc@5  42.19 ( 44.93)
Epoch: [9][ 460/1002]	Time  0.364 ( 0.369)	Data  0.000 ( 0.026)	Loss 4.1468e+00 (3.8759e+00)	Acc@1  17.97 ( 22.46)	Acc@5  40.62 ( 44.95)
Epoch: [9][ 470/1002]	Time  0.365 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.9151e+00 (3.8774e+00)	Acc@1  22.66 ( 22.44)	Acc@5  46.09 ( 44.91)
Epoch: [9][ 480/1002]	Time  0.403 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.9365e+00 (3.8801e+00)	Acc@1  14.84 ( 22.38)	Acc@5  45.31 ( 44.85)
Epoch: [9][ 490/1002]	Time  0.366 ( 0.369

Epoch: [10][   0/1002]	Time  0.761 ( 0.761)	Data  0.626 ( 0.626)	Loss 3.7982e+00 (3.7982e+00)	Acc@1  24.22 ( 24.22)	Acc@5  43.75 ( 43.75)
Epoch: [10][  10/1002]	Time  0.364 ( 0.402)	Data  0.000 ( 0.078)	Loss 3.9081e+00 (3.7886e+00)	Acc@1  18.75 ( 21.59)	Acc@5  42.19 ( 46.66)
Epoch: [10][  20/1002]	Time  0.366 ( 0.385)	Data  0.000 ( 0.053)	Loss 3.8453e+00 (3.7393e+00)	Acc@1  19.53 ( 23.77)	Acc@5  47.66 ( 47.10)
Epoch: [10][  30/1002]	Time  0.365 ( 0.379)	Data  0.000 ( 0.044)	Loss 3.6957e+00 (3.7597e+00)	Acc@1  23.44 ( 23.74)	Acc@5  48.44 ( 46.70)
Epoch: [10][  40/1002]	Time  0.367 ( 0.378)	Data  0.000 ( 0.039)	Loss 3.4099e+00 (3.7391e+00)	Acc@1  25.78 ( 24.14)	Acc@5  53.91 ( 46.99)
Epoch: [10][  50/1002]	Time  0.367 ( 0.376)	Data  0.000 ( 0.037)	Loss 3.9928e+00 (3.7529e+00)	Acc@1  24.22 ( 23.99)	Acc@5  42.19 ( 46.61)
Epoch: [10][  60/1002]	Time  0.361 ( 0.376)	Data  0.000 ( 0.035)	Loss 3.6955e+00 (3.7486e+00)	Acc@1  28.12 ( 24.05)	Acc@5  44.53 ( 46.64)
Epoch: [10][  70/1002]	Time  0.366

Epoch: [10][ 600/1002]	Time  0.365 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.7241e+00 (3.7838e+00)	Acc@1  26.56 ( 23.78)	Acc@5  45.31 ( 46.78)
Epoch: [10][ 610/1002]	Time  0.367 ( 0.369)	Data  0.000 ( 0.026)	Loss 4.1529e+00 (3.7844e+00)	Acc@1  21.88 ( 23.77)	Acc@5  37.50 ( 46.76)
Epoch: [10][ 620/1002]	Time  0.365 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.4824e+00 (3.7840e+00)	Acc@1  25.78 ( 23.80)	Acc@5  48.44 ( 46.80)
Epoch: [10][ 630/1002]	Time  0.364 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.7006e+00 (3.7849e+00)	Acc@1  24.22 ( 23.79)	Acc@5  45.31 ( 46.79)
Epoch: [10][ 640/1002]	Time  0.364 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.8201e+00 (3.7872e+00)	Acc@1  28.12 ( 23.77)	Acc@5  49.22 ( 46.75)
Epoch: [10][ 650/1002]	Time  0.363 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.5206e+00 (3.7882e+00)	Acc@1  31.25 ( 23.75)	Acc@5  46.09 ( 46.73)
Epoch: [10][ 660/1002]	Time  0.367 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.6478e+00 (3.7892e+00)	Acc@1  21.88 ( 23.74)	Acc@5  45.31 ( 46.73)
Epoch: [10][ 670/1002]	Time  0.365

Epoch: [11][ 180/1002]	Time  0.364 ( 0.371)	Data  0.000 ( 0.029)	Loss 3.6047e+00 (3.6198e+00)	Acc@1  23.44 ( 25.93)	Acc@5  53.91 ( 49.65)
Epoch: [11][ 190/1002]	Time  0.363 ( 0.371)	Data  0.000 ( 0.029)	Loss 3.7212e+00 (3.6237e+00)	Acc@1  26.56 ( 25.87)	Acc@5  46.09 ( 49.54)
Epoch: [11][ 200/1002]	Time  0.367 ( 0.371)	Data  0.000 ( 0.029)	Loss 3.6437e+00 (3.6254e+00)	Acc@1  22.66 ( 25.79)	Acc@5  48.44 ( 49.38)
Epoch: [11][ 210/1002]	Time  0.366 ( 0.371)	Data  0.000 ( 0.028)	Loss 3.4813e+00 (3.6274e+00)	Acc@1  27.34 ( 25.78)	Acc@5  53.12 ( 49.34)
Epoch: [11][ 220/1002]	Time  0.367 ( 0.371)	Data  0.000 ( 0.028)	Loss 3.9431e+00 (3.6326e+00)	Acc@1  21.09 ( 25.70)	Acc@5  42.97 ( 49.25)
Epoch: [11][ 230/1002]	Time  0.364 ( 0.370)	Data  0.000 ( 0.028)	Loss 3.5247e+00 (3.6311e+00)	Acc@1  25.78 ( 25.73)	Acc@5  51.56 ( 49.27)
Epoch: [11][ 240/1002]	Time  0.369 ( 0.370)	Data  0.000 ( 0.028)	Loss 3.4598e+00 (3.6412e+00)	Acc@1  28.12 ( 25.58)	Acc@5  50.78 ( 49.12)
Epoch: [11][ 250/1002]	Time  0.364

Epoch: [11][ 780/1002]	Time  0.365 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.5865e+00 (3.6809e+00)	Acc@1  21.88 ( 25.13)	Acc@5  49.22 ( 48.61)
Epoch: [11][ 790/1002]	Time  0.364 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.7114e+00 (3.6815e+00)	Acc@1  22.66 ( 25.12)	Acc@5  46.09 ( 48.62)
Epoch: [11][ 800/1002]	Time  0.365 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.3315e+00 (3.6809e+00)	Acc@1  28.12 ( 25.13)	Acc@5  56.25 ( 48.63)
Epoch: [11][ 810/1002]	Time  0.367 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.9374e+00 (3.6825e+00)	Acc@1  21.09 ( 25.12)	Acc@5  42.19 ( 48.61)
Epoch: [11][ 820/1002]	Time  0.362 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.5381e+00 (3.6831e+00)	Acc@1  26.56 ( 25.11)	Acc@5  50.00 ( 48.60)
Epoch: [11][ 830/1002]	Time  0.366 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.9599e+00 (3.6842e+00)	Acc@1  17.97 ( 25.11)	Acc@5  43.75 ( 48.57)
Epoch: [11][ 840/1002]	Time  0.364 ( 0.369)	Data  0.000 ( 0.026)	Loss 3.8458e+00 (3.6840e+00)	Acc@1  29.69 ( 25.10)	Acc@5  49.22 ( 48.56)
Epoch: [11][ 850/1002]	Time  0.372

In [5]:
import torchvision.models as torch_models
import torch
from collections import OrderedDict

model = torch_models.resnet18()
    
checkpoint = torch.load('/home/jtang/Desktop/original_model/checkpoint.pth.tar')